### Imports & Settings
- Descargar e importar dependencias

In [77]:
# Load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dummy baseline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Preprocessing (Pipelines)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

### Load Data
- Cargar el data set
- Primer vistazo a la distribucion

In [60]:
# Get url from file hosted on GitHub
url = "https://raw.githubusercontent.com/ZocoMacc/car-quirks-ml_InnovaLab25/refs/heads/main/data/cars_data.csv"

# Cargar csv en un DataFrame
df = pd.read_csv(url)

# Explorar la estructura del DataFrame (Sanity Check)
df.info()
df.shape
# df.isna().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    10000 non-null  object 
 1   year                    10000 non-null  int64  
 2   selling_price           10000 non-null  int64  
 3   km_driven               10000 non-null  int64  
 4   fuel                    10000 non-null  int64  
 5   combustible_estimado_l  10000 non-null  float64
 6   seller_type             10000 non-null  int64  
 7   transmission            10000 non-null  int64  
 8   owner                   10000 non-null  int64  
 9   tipo_carroceria         10000 non-null  int64  
 10  potencia_motor_hp       10000 non-null  int64  
 11  nivel_seguridad         10000 non-null  float64
 12  calidad_auto            10000 non-null  object 
 13  score_calidad           10000 non-null  float64
 14  eficiencia_km_l         10000 non-null 

(10000, 15)

In [55]:
# Ver el balance de las clases (84% Media, 10.5% Alta, 5.5% Baja)
df["calidad_auto"].value_counts(normalize=True)

,proportion
calidad_auto,
Media,0.8399
Alta,0.1052
Baja,0.0549


##### Lo que esto nos dice
- Un fuerte desbalanceo de clases
  - "Media" es la mayoria con un 84%
  - "Alta" solo cubre el 10%, y "Baja" solo un 5.5%
- El accuracy de referencia (baseline) es engañosamente alto
  - Un DummyClassifier que siempre predice "Media" deberia de poder predecir 83.99%.
  - Superar este baseline no garantiza que el modelo sea mas precizo detectando "Alta" o "Baja".
- Riesgo de abandono de las clases minoritarias
  - El modelo tiene el riesgo de predecir "Media" la mayoria del tiempo debido al desbalanceo de las clases, lo que podria ignorar "Alta" o "Baja" como opciones.

### Cleaning & Dropping
- Decidir las variables que seran ignoradas por el modelo.
- Si es necesarion, transformar valores en interpretaciones legibles por el modelo o marcarlas con una categoria.

In [61]:
# Duda sobre la variable "score_calidad": Que tanto se correlaciona con "calidad_auto"
# Mapear las categorias con numeros para calcular la correlacion
mapping = {"Baja": 0, "Media": 1, "Alta": 2}
df["calidad_code"] = df["calidad_auto"].map(mapping)

# Calcular la correlacion
corr = df[["score_calidad", "calidad_code"]].corr().iloc[0, 1]

# Imprimir los resultados
print(f"Correlacion entre score_calidad y calidad_auto: {corr:.3f}")

Correlacion entre score_calidad y calidad_auto: 0.740


##### Lo que esto nos dice
- La correlacion entre "score_calidad" y "calidad_auto" no es 1.0, lo que quiere decir que "calidad_auto" no depende al 100% de la variable "score_calidad".
- La variable "score_calidad" va a ser considerada.

#### Extraer marca del nombre
- Usar el nombre y cortarlo de una manera en la que podamos extrar la marca del auto para poder usarlo como variable y ser considerada.

In [62]:
"""
Rifense este rollo porfi
- Extraer el nombre de la marca de cada carro (crar una columna para "brand")
- Investigar si es viable extraer tambien el submodelo del carro (Limited, Sport, AC, ZX, etc)
  - Maybe no nos conviene por que son un chingo pero maybe si si no son tantas quien sabe
  - Si si es viable crear una columna para "submodel"
"""

# Al final se droppea el nombre (ya teniendo la brand y el submodel no lo necesitamos)
df = df.drop(columns=["name"])

#### Convertir columnas de enteros en categorias
- Columnas como "fuel", "seller_type", "transmission", "owner", "tipo_carroceria" tienen asignado un valor entero pero en realidad representan categorias sin orden.
- Convertir estas variables a Pandas' category dtype para mas claridad y eficiencia en la manipulacion de estas variables, tambien es mas seguro hacerlo de esta forma, asi evitamos que estas columnas se interpreten como numeric imputers o scalers.

In [63]:
cat_int_cols = [
    "fuel",
    "seller_type",
    "transmission",
    "owner",
    "tipo_carroceria"
    #"brand"             # caegoria extraida del nombre
]

for col in cat_int_cols:
    df[col] = df[col].astype("category")

#### Droppear columnas innecesarias

In [64]:
# Droppera la columna helper que definimos previemente
df = df.drop(columns=["calidad_code"])
print("Columnas que permanecen en el DataFrame:")
print(df.dtypes)  # Checar las columnas que permanecen

# Define X and y for training
features_to_drop = ["calidad_auto"]
X = df.drop(columns=features_to_drop)
y = df["calidad_auto"]    # Estableciendo el target

# Inspeccionar las variables que permanecen
# print("Columnas a considerar:\n", X.columns.tolist())
print("\nFeature dtypes (para X):")
print(X.dtypes)

Columnas que permanecen en el DataFrame:
year                         int64
selling_price                int64
km_driven                    int64
fuel                      category
combustible_estimado_l     float64
seller_type               category
transmission              category
owner                     category
tipo_carroceria           category
potencia_motor_hp            int64
nivel_seguridad            float64
calidad_auto                object
score_calidad              float64
eficiencia_km_l            float64
dtype: object

Feature dtypes (para X):
year                         int64
selling_price                int64
km_driven                    int64
fuel                      category
combustible_estimado_l     float64
seller_type               category
transmission              category
owner                     category
tipo_carroceria           category
potencia_motor_hp            int64
nivel_seguridad            float64
score_calidad              float64
eficienci

### DummyClassifier & Baseline
- El primer paso es establecer una baseline y asegurarse de que la informacion esta limpia
- El objetivo es confirmar ques este DummyClassifier de como resultado una precision de 83.99%
- Se usara la estrategia de "most_frequent", simplemente el valor que mas se repite sera elegido para la prediccion.
- Este modelo representa el piso (baseline) de qualquier modelo futuro.
- El split para entrenamiento es un 80/20 asegurandose de que cada clase aparece con las misma proporciones.



In [67]:
# Establecer split (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,    # 80/20 split
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Test
# print("Train class proportions:\n", y_train.value_counts(normalize=True))
# print("\nTest class proportions:\n", y_test.value_counts(normalize=True))

# Iniciar DummyClassifier
dummy = DummyClassifier(
    strategy="most_frequent", # Siempre predice la clase mas frecuente ("Media")
    random_state=42
)

dummy.fit(X_train, y_train)     # "Entrena" basado en la clase mas frecuente

# Obtener predicciones
y_pred = dummy.predict(X_test)  # Aplica la regla a cada fila de X_test

# Evaluar accuracy
accuracy = accuracy_score(y_test, y_pred)   # Fraccion de predicciones correctas
print(f"DummyClassifier Accuracy: {accuracy}")

# Visualizar resulatdos de predicciones
print("Classification report:\n")
print(classification_report(
    y_test,
    y_pred,
    digits=4,         # Numero de decimales
    zero_division=0   # Ignorar zero-division errors (debido a strings)
))

# Generar confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

DummyClassifier Accuracy: 0.84
Classification report:

              precision    recall  f1-score   support

        Alta     0.0000    0.0000    0.0000       210
        Baja     0.0000    0.0000    0.0000       110
       Media     0.8400    1.0000    0.9130      1680

    accuracy                         0.8400      2000
   macro avg     0.2800    0.3333    0.3043      2000
weighted avg     0.7056    0.8400    0.7670      2000

Confusion Matrix:
 [[1680    0    0]
 [ 210    0    0]
 [ 110    0    0]]


#### Lo que esto nos dice
- Tenemos un baseline de ~84% accuracy, este es el objetivo a vencer en los proximos modelos.

### Data preprocessing
- Preparar el data set para ser procesado
- Dividir entre variables numericas y categoricas
- Implementar un pipeline para cada tipo de variable
- Usar ColumnTransformer para aplicar los pipelines al DataFrame

#### Identificar columnas en base al dtype

In [69]:
# Datos numericos
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
print("Numeric features:", num_cols)

# Datos categoricos
cat_cols = X.select_dtypes(include=["category"]).columns.tolist()
print("\nCategorical features:", cat_cols)

Numeric features: ['year', 'selling_price', 'km_driven', 'combustible_estimado_l', 'potencia_motor_hp', 'nivel_seguridad', 'score_calidad', 'eficiencia_km_l']

Categorical features: ['fuel', 'seller_type', 'transmission', 'owner', 'tipo_carroceria']


#### Construir el pipeline numerico

In [72]:
num_pipe = Pipeline([
    # Definiendo tecnicas de defensive programming
    ("imputer", SimpleImputer(strategy="median")), # Imputacion de media
    ("scaler", StandardScaler())                   # Standard scaling
])

- El objetivo de la funcion `SimpleImputer()` es remplazar valores faltantes por el valor definido por su argumento (e.g. median).
- El objetivo de la funcion `StandardScaler()` es que despues de una imputacion, el valor numerico es escalado para tener una media = 0 y standard deviation = 1. De esta forma se evitan posibles errores generados por tener diferentes escalas de numeros en las diferentes columnas.

#### Construir el pipeline categorico

In [75]:
cat_pipe = Pipeline([
    # Definiendo tecnicas de defensive programming
    ("imputer", SimpleImputer(strategy="most_frequent")),  # Imputacion de moda
    ("encoder", OneHotEncoder(handle_unknown="ignore"))    # One-hot encoding
])

- `SimpleImputer()` remplaza cualquier valor faltante por el valor mas frecuente (la moda). Esto preserva la distribucion del data set.
- El objetivo del OneHotEncoder es convertir cada columna categorica en las columnas binarias necesarias para representar cada valor categorico unico.
  - `handle_unkwnown="ignore"` es util para que en caso de que aparezca una nueva categoria "vacia", que esta sea representada por una columna entera de puros ceros.

#### Implementacion de ColumnTransformer para aplicar los pipelines

In [79]:
preprocessor = ColumnTransformer(
    transformers=[
        # Aplicar num_pipe a las columnas numericas
        ("nums", num_pipe, num_cols),
        # Aplicar cat_pipe a columnas categoricas
        ("cats", cat_pipe, cat_cols)
    ],
    remainder="drop" # Tirar cualquier columna no listada
)

- `transformers` es un array de tuples `(name, pipeline, columns)` que ira siendo populado mientras `.fit()` corre
  1. `("nums", num_pipe, num_cols)` toma todas las columnas en `num_cols` y aplica el pipeline `num_pipe` y entraga como output el array numerico transformado.
  2. `("cats", cat_pipe, cat_cols)` toma todas las columnas en `cat_cols` y aplica el pipeline `cat_pipe` y entraga como output el array categorico transformado.

#### Inspeccionar el output transformado

In [84]:
# Testear el preprocessor
preprocessor.fit(X_train)

# Ver la forma del array final
X_train_transformed = preprocessor.transform(X_train)
print("Transformed shape:", X_train_transformed.shape)
#print(len(num_cols))

# Imprimir las columnas finales
feature_names = preprocessor.get_feature_names_out()
print("All output features:\n", feature_names)

Transformed shape: (8000, 28)
All output features:
 ['nums__year' 'nums__selling_price' 'nums__km_driven'
 'nums__combustible_estimado_l' 'nums__potencia_motor_hp'
 'nums__nivel_seguridad' 'nums__score_calidad' 'nums__eficiencia_km_l'
 'cats__fuel_0' 'cats__fuel_1' 'cats__fuel_2' 'cats__fuel_3'
 'cats__fuel_4' 'cats__seller_type_0' 'cats__seller_type_1'
 'cats__seller_type_2' 'cats__transmission_0' 'cats__transmission_1'
 'cats__owner_0' 'cats__owner_1' 'cats__owner_2' 'cats__owner_3'
 'cats__owner_4' 'cats__tipo_carroceria_1' 'cats__tipo_carroceria_2'
 'cats__tipo_carroceria_3' 'cats__tipo_carroceria_4'
 'cats__tipo_carroceria_5']


##### Lo que esto nos dice
- En total hay 28 columnas ya con los pipelines aplicados
- Los prefijos `nums_` y `cats_` nos dicen de cual pipeline proviene cada columna
- Estos nombres nos serviran despues para implementacion de funciones o debugging

### Modeling

### Evaluation